In [1]:
import numpy as np
import pandas as pd
import jpx_tokyo_market_prediction
from decimal import ROUND_HALF_UP, Decimal

env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()
path = "../input/jpx-tokyo-stock-exchange-prediction"

In [2]:
stock_list = pd.read_csv(path+'/stock_list.csv')
train_stock_prices = pd.read_csv(path+'/train_files/stock_prices.csv')

In [3]:
def clean_prices(df):
    df = df.copy()
    df = df.drop(["RowId"],axis=1)
    df = df[df["Date"]!="2020-10-01"]
    df["Close"] = df.groupby("SecuritiesCode")["Close"].ffill()
    df["ExpectedDividend"] = df["ExpectedDividend"].fillna(0)
    return df

In [4]:
def clean_data(df,actual):
    df = df.copy()
    df = clean_prices(df)
    df['daily_return'] = (df['Close']-df['Open'])/df['Open']
    df["daily_return"] = df.groupby(["Date"])["daily_return"].apply(lambda x: x.fillna(x.mean()))
    df["Previous Daily Return"] =df.groupby(["SecuritiesCode"])["daily_return"].shift(1)
    df["Previous Daily Return"] =df.groupby(["SecuritiesCode"])["Previous Daily Return"].apply(lambda x: x.fillna(x.mean()))
    df["Rank"] = df.groupby(["Date"])["daily_return"].rank(method='first')-1
    if(actual=='yes'):
        return df
    else:
        df = df.dropna(subset=["Rank"])
        return df

In [5]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    train_stock_prices["Date"] = pd.to_datetime(train_stock_prices["Date"])
    train_stock_prices = train_stock_prices[train_stock_prices["Date"]>"2020"].reset_index(drop=True)
    prices["Date"] =  pd.to_datetime(prices["Date"])
    train_stock_prices = pd.concat([train_stock_prices,prices],ignore_index=True)
    data = clean_data(train_stock_prices,'yes')
    data = data.join(stock_list[["SecuritiesCode","MarketCapitalization"]].set_index("SecuritiesCode"),on="SecuritiesCode")
    data["mkt_cap_rank"] = data.groupby(["Date"])["MarketCapitalization"].rank(method="first")-1
    data["is_not_small_cap"] = data.groupby(["Date"])["mkt_cap_rank"].apply(lambda x: ((x/x.max())>=0.3).astype(int))
    data["above_median_small_cap_rank"]=data.groupby(["Date"]).apply(lambda x: x["Rank"]>x[x["is_not_small_cap"]==0]["Rank"].median()).astype(int).values
    data.loc[(data["above_median_small_cap_rank"]==1)&(data["is_not_small_cap"]==0),["is_not_small_cap"]]=3
    data = data.sort_values(["Date",'Rank',"is_not_small_cap"]).reset_index(drop=True)
    data["Rank"]=data.groupby(["Date"])["Rank"].transform(lambda x: np.linspace(x.min(),x.max(),x.count()))
    data = data.set_index(["Date","SecuritiesCode"])
    data["Rank"] = data["Rank"].astype("int")

    sample_prediction["Rank"]=sample_prediction.apply(lambda x: data.loc[(x["Date"],x["SecuritiesCode"])]["Rank"] , axis=1)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
